***
### <span style='color:green'> ME Algorithm  &emsp;&emsp; Feb, 2024 </span>
### <span style='color:Blue'> Phase 3 </span>
### <p> Yan-Bin Chen (陳彥賓) &emsp; yanbin@stat.sinica.edu.tw </p>
### <p> Institute of Statistical Science, Academia Sinica, Taipei, Taiwan.</p>  
---

# Input

In [ ]:
import pickle
import numpy as np
import collections
import pandas
import itertools
from itertools import chain
import random 
import time
import datetime

In [ ]:
# Input
TRIALS    = 1
timestr   = ''
#Region_Index_loc = 6 # column location; Others is 6
REGION_INDEX_LOC = 4 # column location; MNIST is 4

PATH1='../data/Plant_seedinds.txt'
PATH2='../data/Plant_bilabels.txt'
PATH3='../data/Plant_seedinds_neighborregions.txt'
PATH4='../data/ResNet18_PlantDisease_45K_Spec200.csv'
PATH5='../data/Plant_embeddings.pickle'
#PATH5='../phase3_data/ResNet18_PlantDisease_45K_Labels.csv'
#=================================================================

# (1)PATH1
df = pandas.read_csv(PATH1, header=None, delimiter = "\t")
region = df.to_numpy().T[0]
NUM_region = len(region)

# (2)PATH2
df = pandas.read_csv(PATH2, header=None, delimiter = "\t")
cen = df.to_numpy()

print('region:', region)
print('region amount:', len(region))


# (4)PATH4. Have to be here. The following neighboring process needs this information
df = pandas.read_csv(PATH4)
#1213 add auto judge
all_region_index = df.to_numpy().T[REGION_INDEX_LOC].astype(int)

# Get neighbors

In [ ]:
if (PATH3):
    df = pandas.read_csv(PATH3, delim_whitespace=' ', header=None,  index_col=0)
    neighbors = df.to_numpy()
    NUM_NEI=df.shape[1]
    
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    print(neighbors)

# filter out duplicated ones
    test_list=list(chain.from_iterable(neighbors))
    res2=[]
    [res2.append(n) for n, i in enumerate(test_list) if i in test_list[:n]]

    res2.reverse()
    neighbors=neighbors.tolist()
    [neighbors[x//NUM_NEI].pop(x%NUM_NEI) for x in res2]
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    display(neighbors)

In [ ]:
# combine seed regions and neighbors
if (PATH3):
    reg_nei=[]
    for i in range(len(neighbors)):
        a=[region[i]]
        b=neighbors[i]
        if len(b):
            c=list(np.concatenate((a,b),axis=0))
        else:
            c=a.copy()
        reg_nei.append(c)
else:
    reg_nei=region.copy()

reg_nei

In [ ]:
# collect images from target region and neighboring regions
# input : region, NUM_region, cen, all_region_index, neighbors
# output: region_image: to save image indices corresponding to seed regions.
#         region_answer: to save answer

region_image_before=[]
region_image=[]
region_image_pure=[]
for i in range(NUM_region):
    
    
    #(1)neighbor  nei
    if (PATH3):
        addr_nei=[]
        for j in range(len(neighbors[i])):
            addr_nei=addr_nei+list(np.where(all_region_index==neighbors[i][j])[0])
            #check whether it has duplicates
            if (len(addr_nei) != len(set(addr_nei))):
                print("neighbor duplicate at i=",i,"j=",j)
                addr_nei=list(set(addr_nei))

    #(2)original
    addr=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    if (PATH3):
        addr=addr+addr_nei
    #check whether it has duplicates
    if (len(addr) != len(set(addr))):
        print("** original duplicate at i=",i,";region",region[i],";duplicate size",len(addr)-len(set(addr)))
        addr=list(set(addr))
    region_image.append(addr)

    #(3)original before centroid (this is only for check, rather than for main codes)
    addr_before=list(np.where(all_region_index==region[i])[0])
    if (PATH3):
        addr_before=addr_before+addr_nei
    #check whether it has duplicates
    if (len(addr_before) != len(set(addr_before))):
        print("** same duplicate situation")
        addr_before=list(set(addr_before))
    region_image_before.append(addr_before)

    #(4)pure
    addr_pure=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    region_image_pure.append(addr_pure)


with open('../data/' + timestr + 'region_for_phase5.pickle', 'wb') as f:
    pickle.dump([region, reg_nei, region_image_pure, region_image,], f)

In [ ]:
# ==== test_array ====
with open(PATH5, 'rb') as f:
    test_array, test_label_answer = pickle.load(f)
print("test_array:",np.shape(test_array))


# 1213 add auto judge /255
test_array = np.expand_dims(test_array, axis = -1)
test_array /= 255
print("training size:",len(np.array(list(chain.from_iterable(region_image)))))

# CNN original

In [ ]:
#======================================================

#1213 add auto judge
from CNN_Modules_1D import ME_CNN

# ==== CNN Original =====
ROUND_start = time.time()

for n in range(5):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    p1=0
    p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    # 1213
    # fill up training array
    W           = np.shape(test_array[0])[0]
    train_array = np.zeros((len(Input_img), W), dtype=float)
    for i in range (len(Input_img)):
        train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n0_R' + str(p1) + '+R'+ str(p2) +'_trial' + str(n)+'.pickle'  #extra_original
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)

In [ ]:
#==== shift label =======
N=5
Original_result=[]
Original_prob=[]
for i in range(N):
    with open('./' + timestr + '(classes=' + str(NUM_region) + ')_n0_R0+R0_trial' + str(i)+ '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)
    Original_result.append(label_B)

    #(2)
    Original_prob.append(label_B_prob)
print(np.shape(Original_result), np.shape(Original_prob))

# save pickle
with open('./' + timestr + 'results_of_original.pickle', 'wb') as f:
    pickle.dump([Original_result, Original_prob], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_original.mat', {'result_for_original':Original_result, 'prob_for_original':Original_prob})

# CNN combination and removal

In [ ]:
#====CNN combination c(20,2)======
comb=[]
for subset in itertools.combinations(range(NUM_region), 2):
    comb.append(subset)
NUM_comb=len(comb)
display(NUM_comb)            

for n in range(NUM_comb+1):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    if (n > 0):
        p1=comb[n-1][0]
        p2=comb[n-1][1]
        region[p1]=region[p1]+region[p2]
        region.pop(p2)
        selected_region.pop(-1)
    else:
        p1=0
        p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    #1213
    # fill up training array
    W           = np.shape(test_array[0])[0]
    train_array = np.zeros((len(Input_img), W), dtype=float)
    for i in range (len(Input_img)):
        train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n' + str(n) + '_R' + str(p1) + '+R'+ str(p2) +'.pickle'
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)



# ==== CNN Removal =====
for n in range(NUM_region):
    region=region_image.copy()   #reset
    selected_region = list(range(NUM_region))
    selected_region.pop(n)

    print("n=", n)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    #1213
    # fill up training array
    W           = np.shape(test_array[0])[0]
    train_array = np.zeros((len(Input_img), W), dtype=float)
    for i in range (len(Input_img)):
        train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_Remove' + str(n) +'.pickle' 
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)
  

In [ ]:
# ====shift label=====
#=== combination =====
comb=[]
for oneset in itertools.combinations(range(NUM_region), 2):
    comb.append(oneset)
NUM_comb=len(comb)

Merged_result=[]
Merged_prob=[]
Merged_prob_label=[]

for n in range(NUM_comb):
    label = list(range(NUM_region))
    p1=comb[n][0]
    p2=comb[n][1]

    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_n'+str(n+1)+'_R'+str(p1)+'+R'+str(p2)+'.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)shift label index
    for p in reversed(range(p2,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Merged_result.append(label_B)

    #(2)
    Merged_prob.append(label_B_prob)

    #(3)
    label.pop(p2)
    Merged_prob_label.append(label)

print(np.shape(Merged_result), np.shape(Merged_prob), np.shape(Merged_prob_label))

# save pickle
with open('./' + timestr + 'results_of_combination.pickle', 'wb') as f:
    pickle.dump([comb, Merged_result, Merged_prob, Merged_prob_label], f)

# save mat
from scipy.io import savemat
if (np.shape(Merged_prob)[0]<=300):
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_for_merge':Merged_prob,'prob_label_for_merge': Merged_prob_label})
    print("normal size = ", np.shape(Merged_prob))
else:
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_label_for_merge': Merged_prob_label})
    savemat('./' + timestr + 'results_of_combination_prob1.mat', {'prob_for_merge1':Merged_prob[:200]})
    savemat('./' + timestr + 'results_of_combination_prob2.mat', {'prob_for_merge2':Merged_prob[200:400]})
    savemat('./' + timestr + 'results_of_combination_prob3.mat', {'prob_for_merge3':Merged_prob[400:]})
    print("large size = ", np.shape(Merged_prob))

    
    
    
#======== removal ===========
Removal_result=[]
Removal_prob=[]
Removal_prob_label=[]

for n in range(NUM_region):
    label = list(range(NUM_region))    
    #reset
    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_Remove' + str(n) + '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    for p in reversed(range(n,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Removal_result.append(label_B)

    #(2)
    Removal_prob.append(label_B_prob)

    #(3)
    label.pop(n)
    Removal_prob_label.append(label)

print(np.shape(Removal_result), np.shape(Removal_prob), np.shape(Removal_prob_label))

# save pickle
with open('./' + timestr + 'results_of_removal.pickle', 'wb') as f:
    pickle.dump([Removal_result, Removal_prob, Removal_prob_label], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_removal.mat', {'result_for_removal':Removal_result,'prob_for_removal':Removal_prob, 'prob_label_for_removal':Removal_prob_label})

In [ ]:
print("done")